# How to run models simultaneously

*Jorge Anais*  
*16 August 2020*  

In this document I will describe briefly how to run petitRADTRANS for different physical parameters using multiprocessing module.


In [2]:
import multiprocessing as mp
import numpy as np
from petitRADTRANS import nat_cst as nc
from petitRADTRANS import Radtrans
from platmy import tools
from platmy.model import Planet

To select the respective `abundances.inp` file I use a small function defined in `platmy/tools.py`.

In [3]:
abund_type = 'std'
tools.set_abundance_file(abund_type)

Then, using lists, we define all the basic parameters for each planet, including a name, mass, radius, temperature and pressure.

In [4]:
names = ['Jupiter-like', 'HD1397b', 'TOI172b', 'TOI677b']
masses = np.array([1., 0.367, 5.42, 1.236]) * nc.m_jup
radii = np.array([1., 1.023, 0.965, 1.170]) * nc.r_jup_mean
temps_equ = np.arange(500., 2001., 500.)  # grid of temperatures
pressure = np.logspace(-6, 2, 100)  # pressure (same for all planets)
p0 = 0.10  # bar

From those parameters, we build the temperature, pressure and abundances (mass_fractions) structures for each planet.

For convinience, I store all the relevant info for a single planet as an object of the class `Planet` (in `platmy/models.py`). During the construction of the object, there is a method that call `easy_chem` program that builds the abundances structure from the (P,T) structure. Notice that we are not using a constant value for abundances, instead, abundances are computed for every layer depending on the pressure and temperature of the layer. 

This step may take a while to complete.

In [5]:
planets = [Planet(name=n, radius=r, temp_equ=t, mass=m, pressure=pressure)
           for t in temps_equ
           for n, m, r in zip(names, masses, radii)]


Then we define an instance of the Radtrans object with all the species that we want to consider for the computations of the spectra. Don't forget to check that all the species are included in the `abundances.ino` file.

In [6]:
atmosphere = Radtrans(line_species=['C2H2', 'CH4', 'CO', 'CO2', 'H2', 'H2O', 'H2S', 'HCN', 'K', 'NH3', 'Na', 'OH', 'PH3', 'TiO', 'VO'],
                      rayleigh_species=['H2', 'He'],
                      continuum_opacities=['H2-H2', 'H2-He'],
                      cloud_species=['Mg2SiO4(c)_cd'],
                      wlen_bords_micron=[0.3, 15],  # max wlen range 0.11 to 250 microns for 'c-k' mode. Ram intensive.
                      mode='c-k')


  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
 Done.



We set the haze and pcloud parameters. If you don't want to incorporate them in the model set to None.

In [7]:
haze_factor = 10
pcloud = 0.01
description = f'A X planet with {abund_type} abundances and haze_factor={haze_factor} and pcloud={pcloud}'

Also, we add some Condensate clouds parameters to our models. As before, set to None if you don't want to use it.

In [8]:
radius = {}
radius['Mg2SiO4(c)'] = 0.00005 * np.ones_like(pressure) # I.e. a 0.5-micron particle size (0.00005 cm)
sigma_lnorm = 1.05

Now we run our models in parallel using the multiprocessing library. For that we need a function, in this case `worker()` (in `platmy.tools.py`), and a grid of parameters.

We construct the grid of parameters, one per planet, as follows:

In [9]:
# Grid of params
models = [(atmosphere, pl, p0, sigma_lnorm, radius, pcloud, haze_factor, description) for pl in planets]

We execute this function in parallel. Because it could be very ram intensive if you run several models at the same time (at least for my PC), we set the numer of parallel process to 3.

In [11]:
with mp.Pool(3) as pool:
    results = pool.starmap(tools.worker, models)

Once completed, you should find the files with the models and plots in the `datagen/` and `plots/` folders respectively.